## Modelo de datos de la estación de esquí
### Este modelo de datos representa una estación de esquí que tiene una gran colección de datos para cada telesilla recopilada diariamente.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de todos los datos de un remonte determinado o complejo general, dinámico y estático, facilitada por una tabla llamada SkiLifts.

* Recuperación de todos los datos dinámicos (incluidos los transportes únicos, la cobertura de nieve, el peligro de avalancha y el estado del ascensor) para un telesilla o el centro vacacional en una fecha específica, facilitada por una tabla llamada SkiLifts.

* Recuperación de todos los datos estáticos (incluyendo si el transporte es solo para experimentados, pies verticales que el transporte se eleva y tiempo de conducción en el transporte) para un telesilla específico, facilitado por una mesa llamada SkiLifts.

* Recuperación de la fecha de los datos registrados para un remonte específico o el complejo general ordenado por usuarios individuales totales, facilitada por el índice secundario global de la tabla SkiLifts llamado SkiLiftsByRiderss.

In [2]:
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from time import sleep

In [3]:
dt = DynamoTable()
try:
    dt.select_table('SkiLifts')
    print(dt)
except:
    dt.create_table(
        table_name='SkiLifts',
        partition_key='Lift',
        partition_key_type='S',
        sort_key="Metadata",
        sort_key_type="S"
)

Table created successfully!


In [4]:
dt.load_json("ski.json")

In [5]:
dt.bill_mode

'PROVISIONED'

In [6]:
dt.create_global_secondary_index(
    att_name="Lift",
    att_type="S",
    sort_index="TotalUniqueLiftRiders",
    sort_type="S",
    i_name="SkiLiftsByRiders",
    proj_type=["Attributes"]
)

In [7]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

Global secondary index created.


In [27]:
dt.list_gsi

['SkiLiftsByRiders']

In [5]:
dt.query_partiql(
    query="SELECT * FROM SkiLifts WHERE Metadata = ?",
    parameters=["Static Data"],
)

,VerticalFeet,Metadata,LiftTime,ExperiencedRidersOnly,Lift
0,900,Static Data,5:45,True,Lift 23
1,1300,Static Data,7:30,False,Lift 3
2,1500,Static Data,9:00,False,Lift 16


In [11]:
dt.query_partiql(
    query="SELECT Lift, LiftStatus, Metadata FROM SkiLifts WHERE Metadata <= ?",
    parameters=["02/01/20"]
)

,LiftStatus,Metadata,Lift
0,Closed,01/01/20,Lift 23
1,Open,02/01/20,Lift 23
2,Open,01/01/20,Lift 3
3,Open,02/01/20,Lift 3
4,NaN,01/01/20,Resort Data
5,NaN,02/01/20,Resort Data
6,Open,01/01/20,Lift 16
7,Open,02/01/20,Lift 16
